In [1]:
import pandas as pd
from scipy.spatial.distance import squareform, pdist

In [2]:
import logging
import json
import boto3
from botocore.exceptions import ClientError

logging.basicConfig(format='%(asctime)s %(levelname)s %(process)d --- %(name)s %(funcName)20s() : %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)


def download_object(bucket: str, key: str):
    s3_client = boto3.client('s3')
    try:
        s3_response_object = s3_client.get_object(Bucket=bucket, Key=key)
        object_content = s3_response_object['Body'].read().decode('utf-8')
        json_content = json.loads(object_content)
        return json_content
    except ClientError as e:
        logging.error(e)


In [3]:
l = download_object('snowboard-finder', 'raw/mens.json')

18-Jun-21 16:00:22 INFO 84853 --- botocore.credentials                 load() : Found credentials in shared credentials file: ~/.aws/credentials


In [19]:
df = pd.json_normalize(l)
df.head()

,id,ratings.Overall Rating,ratings.Riding Style,ratings.Riding Level,ratings.Fits Boot size (US),ratings.Manufactured in,ratings.Shape,ratings.Camber Profile,ratings.Stance,ratings.Approx. Weight,...,ratings.Turn,ratings.Skidded,ratings.Flex,ratings.Buttering,ratings.Edge,meta_data.name,meta_data.gender,meta_data.url,meta_data.price,meta_data.image_url
0,b'pcQjfBFdQmn6Aqep1PKrAJ8tpZY=',Liked it!,All Mountain Freestyle,Beginner - Expert,8-10,Tunisia by Nidecker,True Twin,Hybrid Camber,Centered,Feels Normal,...,Initiation,Turns,Medium,Semi-Easy,Hold,alloy-b-bomb-2016-2020-snowboard-review,mens,https://thegoodride.com/snowboard-reviews/allo...,$549,https://thegoodride.com/assets/lg-gallery/allo...
1,b'WdfFXZW/dvMLdhtMZs3ihVDCRHI=',,Freestyle,Intermediate - Expert,8-10,Tunisia by Nidecker,True Twin,Hybrid Camber,Centered,Feels Normal,...,Initiation,Turns,Medium/Soft,Easy,Hold,alloy-b-bomb-gt-2020-snowboard-review,mens,https://thegoodride.com/snowboard-reviews/allo...,$0,https://thegoodride.com/assets/lg-gallery/allo...
2,b'GLXwggrHSpW/E/vBriKPps5WPmE=',Pretty Good,Freeride,Expert,8-10,Tunisia by Nidecker,Tapered Directional,Traditional Camber,Setback over 20mm,Feels Heavy,...,Initiation,Turns,Stiff,Hard,Hold,alloy-d-o-carbon-2019-2020-snowboard-review,mens,https://thegoodride.com/snowboard-reviews/allo...,$999,https://thegoodride.com/assets/lg-gallery/allo...
3,b'vyqJS+sM6x2avNeKZCtTt6aTMlA=',,Alternative Freeride,Advanced - Expert,8-10,Tunisia by Nidecker,Tapered Directional,Directional Camber,Setback over 20mm,Feels Normal,...,Initiation,Turns,Medium/Stiff,Semi-Hard,Hold,alloy-darwin-flow-2020-snowboard-review,mens,https://thegoodride.com/snowboard-reviews/allo...,$0,https://thegoodride.com/assets/lg-gallery/allo...
4,b'IbVxVGqf7kOWjwmJu3U0lxr16gk=',Liked it!,Freeride,Expert,8-10,Tunisia by Nidecker,Tapered Directional,Mostly Camber,Setback over 20mm,Feels Normal,...,Initiation,Turns,Medium/Stiff,Hard,Hold,alloy-do-7x-2017-snowboard-review,mens,https://thegoodride.com/snowboard-reviews/allo...,$749,https://thegoodride.com/assets/lg-gallery/allo...


In [49]:
df['id'][0]

"b'pcQjfBFdQmn6Aqep1PKrAJ8tpZY='"

In [20]:
sel_cols = ['ratings.Riding Style',
       'ratings.Riding Level', 'ratings.Shape', 'ratings.Camber Profile',
       'ratings.Stance', 'ratings.Approx. Weight', 'ratings.Powder',
       'ratings.Turning', 'ratings.Carving', 'ratings.Speed', 'ratings.Uneven',
       'ratings.Switch', 'ratings.Jumps', 'ratings.Jibbing', 'ratings.Pipe',
       'ratings.On', 'ratings.Turn', 'ratings.Skidded', 'ratings.Flex',
       'ratings.Buttering', 'ratings.Edge']

meta_cols = ['id', 
             'meta_data.name', 
             'meta_data.name',
             'meta_data.gender',
             'meta_data.url',
             'meta_data.price',
             'meta_data.image_url'
            ]

In [21]:
df_train = df[sel_cols]

In [22]:
def distance_function(r, algo='jaccard'):
    dists = pdist(r, algo)
    
    return pd.DataFrame(squareform(dists))

In [24]:
df_dist = distance_function(pd.get_dummies(df_train, columns=sel_cols))

In [40]:
sim_boards = []
for idx, row in df_dist.iterrows():
    
    target = df_dist.iloc[:, idx].nsmallest(11)
    topn_index = target[target !=0].index
#     df_target = pd.merge(df_final['id'][target.index].reset_index(), target.reset_index(), on='index')
#     df_target.columns = ['index', 'id', 'sim_value']
#     sim_dict = pd.Series(df_target['sim_value'].values, index=df_target['id']).to_dict()
#     sim_boards.append(sim_dict)

In [48]:
encode(df['id'][topn_index][6]

SyntaxError: unexpected EOF while parsing (<ipython-input-48-32c8a35aebd9>, line 1)

In [41]:
topn_index

Int64Index([6, 15, 7, 0, 11, 18, 1, 4, 8, 10], dtype='int64')